- try truncated validation in regression

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, cohen_kappa_score
import lightgbm as lgb
from functools import partial
import json
import copy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from hyperopt import hp, tpe, Trials, fmin, space_eval
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")
import random

In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder_cla(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
        #return -mod_qwk(y, X_p, weights=weights)
    
    def fit(self, X, y, random_flg = False):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if random_flg:
            initial_coef = [np.random.uniform(0.4,0.5), np.random.uniform(0.5,0.6), np.random.uniform(0.6,0.7)]
        else:
            initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead') #Powell
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']
    
class OptimizedRounder_reg(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
        #return -mod_qwk(y, X_p, weights=weights)
    
    def fit(self, X, y, random_flg = False):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if random_flg:
            initial_coef = [np.random.uniform(1.0,1.1), np.random.uniform(1.7,1.8), np.random.uniform(2.1,2.2)]
        else:
            initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead') #Powell
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

In [4]:
def eval_qwk_lgb_regr(y_pred, train_t):
    dist = Counter(train_t['accuracy_group'])
    for k in dist:
        dist[k] /= len(train_t)
    
    acum = 0
    bound = {}
    for i in range(3):
        acum += dist[i]
        bound[i] = np.percentile(y_pred, acum * 100)

    def classify(x):
        if x <= bound[0]:
            return 0
        elif x <= bound[1]:
            return 1
        elif x <= bound[2]:
            return 2
        else:
            return 3

    y_pred = np.array(list(map(classify, y_pred)))
    
    return y_pred

# install

In [5]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 17s, sys: 12 s, total: 1min 29s
Wall time: 1min 28s


# Preprocess and Feature engineering

In [6]:
%%time
def encode_title(train, test):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = sorted(list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique()))))
    
    train['type_world'] = list(map(lambda x, y: str(x) + '_' + str(y), train['type'], train['world']))
    test['type_world'] = list(map(lambda x, y: str(x) + '_' + str(y), test['type'], test['world']))
    list_of_type_world = sorted(list(set(train['type_world'].unique()).union(set(test['type_world'].unique()))))
    
    list_of_user_activities = sorted(list(set(train['title'].unique()).union(set(test['title'].unique()))))
    list_of_event_code = sorted(list(set(train['event_code'].unique()).union(set(test['event_code'].unique()))))
    list_of_worlds = sorted(list(set(train['world'].unique()).union(set(test['world'].unique()))))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = sorted(list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index))))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
        
    train["true"] = train["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)
    test["true"] = test["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)

    train["false"] = train["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
    test["false"] = test["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
    
    train["game_complete"] = train["event_data"].apply(lambda x: 1 if "game_completed" in x else 0)
    test["game_complete"] = test["event_data"].apply(lambda x: 1 if "game_completed" in x else 0)
    
    #train["level"] = train["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    #test["level"] = test["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    
    #train["round"] = train["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
    #test["round"] = test["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
               
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_type_world

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode, list_of_type_world = encode_title(train, test)

CPU times: user 1min 48s, sys: 9.44 s, total: 1min 58s
Wall time: 1min 57s


In [7]:
def make_ratio(features, dic):
    total = sum(dic.values())
    if total != 0:
        for key in dic.keys():
            features[str(key)] = features[str(key)] / total
    else:
        pass
    return features

def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()}
    event_code_count = {str(ev): 0 for ev in list_of_event_code}
    title_count = {actv: 0 for actv in list_of_user_activities}
    type_world_count = {str(ev): 0 for ev in list_of_type_world}
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    last_game_time_title = {'lgt_' + title: 0 for title in assess_titles}
    ac_game_time_title = {'agt_' + title: 0 for title in assess_titles}
    ac_true_attempts_title = {'ata_' + title: 0 for title in assess_titles}
    ac_false_attempts_title = {'afa_' + title: 0 for title in assess_titles}
    
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = user_sample.iloc[0]['timestamp']
    miss = 0
    crys_game_true = 0; crys_game_false = 0
    tree_game_true = 0; tree_game_false = 0
    magma_game_true = 0; magma_game_false = 0
    crys_game_acc = []; tree_game_acc = []; magma_game_acc = []
    durations = []
    prev_assess_title = -999
    assess_count = 1
    last_accuracy = -999
    prev_assess_start = -999; prev_assess_end = -999
    real_prev_assess_start = -999; real_prev_assess_end = -999
    real_assess_start = -999; real_assess_end = -999
    complete_games = 0
    no_result_count = 0
    crys_game_level = np.array([]); tree_game_level = np.array([]); magma_game_level = np.array([])
    crys_game_round = np.array([]); tree_game_round = np.array([]); magma_game_round = np.array([])
    
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        if session_type != 'Assessment':
            if session_type == "Game":
                true = session['true'].sum()
                false = session['false'].sum() 
                if session_world == activities_world["CRYSTALCAVES"]:
                    crys_game_true += true
                    crys_game_false += false
                    crys_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #crys_game_level = np.concatenate([crys_game_level, session["level"]], axis=0)
                    #crys_game_round = np.concatenate([crys_game_round, session["round"]], axis=0)
                elif session_world == activities_world["TREETOPCITY"]:
                    tree_game_true += true
                    tree_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #tree_game_level = np.concatenate([tree_game_level, session["level"]], axis=0)
                    #tree_game_round = np.concatenate([tree_game_round, session["round"]], axis=0)
                elif session_world == activities_world["MAGMAPEAK"]:
                    magma_game_true += true
                    magma_game_false += false
                    magma_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                    #magma_game_level = np.concatenate([magma_game_level, session["level"]], axis=0)
                    #magma_game_round = np.concatenate([magma_game_round, session["round"]], axis=0)
                else:
                    pass
                
        if (session_type == 'Assessment') & (test_set or len(session)>1): 
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            assess_start = session.iloc[0,2]
            assess_end = session.iloc[-1,2]
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features = make_ratio(features, user_activities_count)
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features = make_ratio(features, title_eventcode_count)
            features.update(user_world_count.copy()) # appearance of world with duplicates
            features = make_ratio(features, user_world_count)
            features.update(event_code_count.copy())
            features = make_ratio(features, event_code_count)
            features.update(title_count.copy())
            features = make_ratio(features, title_count)
            features.update(type_world_count.copy())
            features = make_ratio(features, type_world_count)
            features.update(last_accuracy_title.copy())
            features.update(last_game_time_title.copy())
            features.update(ac_game_time_title.copy())
            features.update(ac_true_attempts_title.copy())
            features.update(ac_false_attempts_title.copy())
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            ac_true_attempts_title['ata_' + session_title_text] += true_attempts
            ac_false_attempts_title['afa_' + session_title_text] += false_attempts
            last_game_time_title['lgt_' + session_title_text] = session['game_time'].iloc[-1]
            ac_game_time_title['agt_' + session_title_text] += session['game_time'].iloc[-1]
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            features["no_complete_game"] = complete_games
            features["no_result_count"] = no_result_count 
            
            if true_attempts + false_attempts == 0:
                no_result_count += 1
            else:
                real_assess_start = session.iloc[0,2]
                real_assess_end = session.iloc[-1,2]
             
            #features["crys_game_true"] = crys_game_true
            #features["crys_game_false"] = crys_game_false
            #features['crys_game_accuracy'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
            #features["crys_game_accuracy_std"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
            #features["cryslast_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
            #features["tree_game_true"] = tree_game_true
            #features["tree_game_false"] = tree_game_false
            #features['tree_game_accuracy'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
            #features["tree_game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
            #features["tree_last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
            #features["magma_game_true"] = magma_game_true
            #features["magma_game_false"] = magma_game_false
            #features['magma_game_accuracy'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
            #features["magma_game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
            #features["magma_last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
            
            if session_world == activities_world["CRYSTALCAVES"]:
                features["game_true"] = crys_game_true
                features["game_false"] = crys_game_false
                features['game_accuracy'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
                features["last_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
            elif session_world == activities_world["TREETOPCITY"]:
                features["game_true"] = tree_game_true
                features["game_false"] = tree_game_false
                features['game_accuracy'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
                features["last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
            elif session_world == activities_world["MAGMAPEAK"]:
                features["game_true"] = magma_game_true
                features["game_false"] = magma_game_false
                features['game_accuracy'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
                features["last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
            
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            features["prev_assess_title"] = prev_assess_title
            prev_assess_title = session_title
            features["first_assessment"] = 1 if assess_count == 1 else 0
            #features["assess_count"] = assess_count
            assess_count += 1
            features["time_from_start"] = (assess_start - time_first_activity).seconds

            if prev_assess_end == -999:
                features["time_bet_assess"] = -999
            else:
                features["time_bet_assess"] = (assess_start - prev_assess_end).seconds
            prev_assess_start = assess_start
            prev_assess_end = assess_end
            if real_prev_assess_end == -999:
                features["time_bet_real_assess"] = -999
            else:
                features["time_bet_real_assess"] = (real_assess_start - real_prev_assess_end).seconds
            real_prev_assess_start = real_assess_start
            real_prev_assess_end = real_assess_end
            
            if durations == []: #span of timestamp in target assessment
                features['duration_mean'] = 0
                features['duration_std'] = 0
                features['duration_max'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
                features['duration_max'] = np.max(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds)
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            features['last_assess_acc'] = last_accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            last_accuracy = accuracy
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
            
        complete_games += np.sum(session["game_complete"])
        miss += np.sum(session["misses"])
        user_world_count["world_"+str(session_world)] += session.shape[0]
        
        n_of_type_world = Counter(session['type_world']) 
        for key in n_of_type_world.keys():
            type_world_count[str(key)] += n_of_type_world[key]
            
        n_of_title = Counter(session['title']) 
        for key in n_of_title.keys():
            title_count[activities_labels[key]] += n_of_title[key]
            
        n_of_eventcode = Counter(session['event_code']) 
        for key in n_of_eventcode.keys():
            event_code_count[str(key)] += n_of_eventcode[key]
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
        
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1], all_assessments[:-1] # test previous data to incorporate into training
    return all_assessments

In [8]:
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    compiled_val = []

    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
        compiled_train += get_data(user_sample)
    del train
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
        test_data, val_data = get_data(user_sample, test_set=True)
        compiled_test.append(test_data)
        compiled_val += val_data
    del test
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    reduce_val = pd.DataFrame(compiled_val)

    categoricals = ['session_title']
    return reduce_train, reduce_test, reduce_val, categoricals
new_train, new_test, new_val, categoricals = get_train_and_test(train, test)

In [9]:
tmp = new_train[new_train.Game==0].copy()
tmp = tmp[tmp.Activity == 0].copy()
tmp = tmp[tmp.Clip == 0].copy()
tmp = tmp[tmp.Assessment ==0].copy()
remove_train_index = tmp.index
new_train = new_train[~new_train.index.isin(remove_train_index)].copy()

In [10]:
print(new_train.shape)
print(new_test.shape)
print(new_val.shape)

(17577, 563)
(1000, 563)
(2347, 563)


In [11]:
# data augmentation
tmp = new_val[new_val.Game==0].copy()
tmp = tmp[tmp.Activity == 0].copy()
tmp = tmp[tmp.Clip == 0].copy()
tmp = tmp[tmp.Assessment ==0].copy()
remove_val_index = tmp.index
add_val = new_val[~new_val.index.isin(remove_val_index)].copy()

#tmp = add_val.installation_id.value_counts().reset_index(drop=False) # include some part of new_test installation_id
#val_id = list(tmp[tmp.installation_id >= 20]["index"])
#add_val = add_val[add_val.installation_id.isin(val_id)]

mod_train = pd.concat([new_train, add_val], ignore_index=True)
mod_train.shape

(19906, 563)

# Feature selection

In [12]:
def exclude(reduce_train, reduce_test, features):
    to_exclude = [] 
    ajusted_test = reduce_test.copy()
    for feature in features:
        if feature not in ['accuracy_group', 'installation_id', 'session_title', 'hightest_level']:
            data = reduce_train[feature]
            train_mean = data.mean()
            data = ajusted_test[feature] 
            test_mean = data.mean()
            try:
                ajust_factor = train_mean / test_mean
                if ajust_factor > 10 or ajust_factor < 0.1:# or error > 0.01:
                    to_exclude.append(feature)
                    print(feature)
                else:
                    ajusted_test[feature] *= ajust_factor
            except:
                to_exclude.append(feature)
                print(feature)
    return to_exclude, ajusted_test
features = [i for i in new_train.columns if i not in ["game_session"]]
to_exclude, ajusted_test = exclude(new_train, new_test, features)

Air Show_4080
Bottle Filler (Activity)_2010
Bubble Bath_4080
Bubble Bath_4090
Bug Measurer (Activity)_4080
Cart Balancer (Assessment)_4080
Chest Sorter (Assessment)_4080
Crystals Rule_2010
Dino Dive_4080
Dino Drink_4080
Egg Dropper (Activity)_4080
Fireworks (Activity)_4080
Happy Camel_4080
Leaf Leader_4080
Mushroom Sorter (Assessment)_4080
Pan Balance_2010
Pan Balance_4080
Sandcastle Builder (Activity)_2010
Scrub-A-Dub_4080
Watering Hole (Activity)_2010
acc_Cart Balancer (Assessment)


# classification

In [13]:
def feature_selection(train):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_train = X_train[sorted(X_train.columns.tolist())]

    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.9605425291685099,'bagging_freq': 4,'colsample_bytree': 0.6784238046856443,
 'feature_fraction': 1,'learning_rate': 0.017891320270412462,'max_depth': 7,
 'min_data_in_leaf': 8,'min_sum_hessian_in_leaf': 17,'num_leaves': 17}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    features_list = [i for i in X_train.columns if i != "installation_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id'],axis=1)
    
        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        X_test2 = X_test2.drop(['installation_id'],axis=1)
            
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
            num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
        test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid.iloc[test_index] = test_predict.reshape(X_test2.shape[0], 1)
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
                
    print("logloss = \t {}".format(log_loss(y_train, valid)))
    print("ROC = \t {}".format(roc_auc_score(y_train, valid)))
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    return feature_importance_df
#df_for_classification = feature_selection(new_train)

In [14]:
def accuracy_class(train, test, fea, select_flg):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude ]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    if select_flg == True:
        X_train = X_train[fea]
    X_train = X_train[sorted(X_train.columns.tolist())]

    X_test = test.drop(["installation_id","accuracy_group"], axis=1)
    X_test = X_test.drop(remove_features, axis=1)
    if select_flg == True:
        X_test = X_test[fea]
    X_test = X_test[sorted(X_test.columns.tolist())]
    print(X_test.shape[1])
    
    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.9605425291685099,'bagging_freq': 4,'colsample_bytree': 0.6784238046856443,
 'feature_fraction': 1,'learning_rate': 0.017891320270412462,'max_depth': 7,
 'min_data_in_leaf': 8,'min_sum_hessian_in_leaf': 17,'num_leaves': 17}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id'],axis=1)
    
        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        X_test2 = X_test2.drop(['installation_id'],axis=1)
        if select_flg == True:
            X_train2 = X_train2[fea] 
            X_test2 = X_test2[fea]
            
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
            num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
        test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        models.append(clf)
        valid.iloc[test_index] = test_predict.reshape(X_test2.shape[0], 1)
                
    print("logloss = \t {}".format(log_loss(y_train, valid)))
    print("ROC = \t {}".format(roc_auc_score(y_train, valid)))
    print('Accuracy score = \t {}'.format(accuracy_score(y_train, np.round(valid))))
    print('Precision score = \t {}'.format(precision_score(y_train, np.round(valid))))
    print('Recall score =   \t {}'.format(recall_score(y_train, np.round(valid))))
    print('F1 score =      \t {}'.format(f1_score(y_train, np.round(valid))))
    print(confusion_matrix(y_train, np.round(valid)))
    pred_value = np.zeros([X_test.shape[0]])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return pred_value, valid

#tmp = df_for_classification.sort_values("Cv", ascending = True).reset_index(drop=True).copy()
#feat = tmp[tmp.index <= 120]["Feature"]
#feat = []
#pred_value, valid = accuracy_class(new_train, new_test, feat, False)
#pred_value, valid = accuracy_class(mod_train, new_test, feat, False)

In [15]:
#best_score = 0
#for i in range(10):
#    optR = OptimizedRounder_cla()
#    #optR.fit(np.array(valid).reshape(-1,), new_train.accuracy_group, random_flg=True)
#    optR.fit(np.array(valid).reshape(-1,), mod_train.accuracy_group, random_flg=True)
#    coefficients = optR.coefficients()
#    final_valid_pred = optR.predict(np.array(valid).reshape(-1,), coefficients)
    #score = qwk(new_train.accuracy_group, final_valid_pred)
    #score = qwk(mod_train.accuracy_group, final_valid_pred)
#    print(i, np.sort(coefficients), score)
#    if score > best_score:
#        best_score = score
#        best_coefficients = coefficients
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort(best_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort([0.31635244, 0.54903181, 0.74679975])) + [np.inf], labels = [0, 1, 2, 3])

#sample_submission["accuracy_group"] = final_test_pred.astype(int)
#sample_submission.to_csv('submission.csv', index=False)
#sample_submission["accuracy_group"].value_counts(normalize = True)

# truncated version

In [16]:
def feature_selection_mod(train):
    X_train = train.drop(['accuracy_group'],axis=1) 
    y_train = train.accuracy_group.copy()
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_train = X_train[sorted(X_train.columns.tolist())]

    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.9605425291685099,'bagging_freq': 4,'colsample_bytree': 0.6784238046856443,
 'feature_fraction': 1,'learning_rate': 0.017891320270412462,'max_depth': 7,
 'min_data_in_leaf': 8,'min_sum_hessian_in_leaf': 17,'num_leaves': 17}

    valid = pd.DataFrame(np.zeros([X_train.shape[0]]))
    features_list = [i for i in X_train.columns if i != "installation_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    random_try = 10
    for try_time in range(random_try):
        valid = np.array([])
        real = np.array([])
        for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
            print("Fold "+str(i+1))
            X_train2 = X_train.iloc[train_index,:]
            y_train2 = y_train.iloc[train_index]
            X_train2 = X_train2.drop(['installation_id'],axis=1)
    
            X_test2 = X_train.iloc[test_index,:]
            y_test2 = y_train.iloc[test_index]
            
            X_test2, idx_val = get_random_assessment(X_test2)
            #tmp_target = X_test2.loc[idx_val]["past_target"]
            X_test2.drop(['installation_id'], inplace=True, axis=1) # 'past_target'
            y_test2 = y_test2.loc[idx_val]
            print("After truncation:", (X_test2.shape, y_test2.shape))
            
            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
            valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], )
            valid = np.concatenate([valid, valid_predict])
            real = np.concatenate([real, y_test2])
            feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
                
        print("logloss = \t {}".format(log_loss(real, valid)))
        print("ROC = \t {}".format(roc_auc_score(real, valid)))
        feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
        feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    return feature_importance_df
#df_for_classification = feature_selection_mod(new_train)

In [17]:
def get_random_assessment(reduce_train):
    used_idx = []
    for iid in tqdm(set(reduce_train['installation_id']), miniters=200):
        list_ = list(reduce_train[reduce_train['installation_id'] == iid].index)
        cur = random.choices(list_, k=1)[0]
        used_idx.append(cur)
    reduce_train_t = reduce_train.loc[used_idx]
    return reduce_train_t, used_idx

def accuracy_class_mod(train, test, fea, select_flg):
    #X_train = train.drop(["accuracy_group"],axis=1) 
    y_train = train.accuracy_group.copy()
    X_train = train.rename(columns={"accuracy_group": "past_target"})
    y_train.loc[y_train <=1] = 0
    y_train.loc[y_train >=2] = 1
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    if select_flg == True:
        X_train = X_train[fea]
    X_train = X_train[sorted(X_train.columns.tolist())]

    X_test = test.drop(["installation_id","accuracy_group"], axis=1)
    X_test = X_test.drop(remove_features, axis=1)
    if select_flg == True:
        X_test = X_test[fea]
    X_test = X_test[sorted(X_test.columns.tolist())]
    
    n_folds = 10
    skf=GroupKFold(n_splits = n_folds)
    models = []
    lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','bagging_fraction': 0.5698056418890787,'bagging_freq': 4,
 'colsample_bytree': 0.37564408454469,'learning_rate': 0.015433389422506185,'max_depth': 8,
 'min_data_in_leaf': 51,'min_sum_hessian_in_leaf': 10,'num_leaves': 48}
    random_try = 20
    mean_qwk_score = 0
    # model learning ---------------------------
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id', 'past_target'],axis=1)
    
        for try_time in range(random_try):
            X_test2 = X_train.iloc[test_index,:]
            y_test2 = y_train.iloc[test_index]
            
            # random truncation ------------------------------------------
            X_test2, idx_val = get_random_assessment(X_test2)
            tmp_target = X_test2.loc[idx_val]["past_target"]
            X_test2.drop(['installation_id', 'past_target'], inplace=True, axis=1)
            y_test2 = y_test2.loc[idx_val]
            print("After truncation:", (X_test2.shape, y_test2.shape))
            # ------------------------------------------------------------
            
            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500,categorical_feature = categoricals)
            valid = np.array(clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], ))
            real = np.array(y_test2)
            target = np.array(tmp_target)
            
            models.append(clf)
            print("logloss = \t {}".format(log_loss(real, valid)))
            print("ROC = \t {}".format(roc_auc_score(real, valid)))
        
            # threshold optimization --------------
            best_score = 0
            for j in range(20):
                optR = OptimizedRounder_cla()
                optR.fit(np.array(valid).reshape(-1,), target, random_flg=True)
                coefficients = optR.coefficients()
                final_valid_pred = optR.predict(np.array(valid).reshape(-1,), coefficients)
                score = qwk(target, final_valid_pred)
                print(j, np.sort(coefficients), score)
                if score > best_score:
                    best_score = score
                    best_coefficients = coefficients
            mean_qwk_score += best_score / (random_try * n_folds)
            if try_time == 0 and i == 0:
                final_coefficients = np.sort(best_coefficients) / (random_try * n_folds)
            else:
                final_coefficients += np.sort(best_coefficients) / (random_try * n_folds)
            
    print("MEAN QWK = \t {}".format(mean_qwk_score))
    # test prediction  ------------------------
    pred_value = np.zeros([X_test.shape[0]])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return pred_value, valid, final_coefficients

#tmp = df_for_classification.sort_values("Cv", ascending = True).reset_index(drop=True).copy()
#feat = tmp[tmp.index <= 120]["Feature"]
#feat = []
#pred_value, valid, final_coefficients = accuracy_class_mod(new_train, new_test, feat, False)
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort(final_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort([0.32229148, 0.51887455, 0.77529457])) + [np.inf], labels = [0, 1, 2, 3])

#sample_submission["accuracy_group"] = final_test_pred.astype(int)
#sample_submission.to_csv('submission.csv', index=False)
#sample_submission["accuracy_group"].value_counts(normalize = True)

In [18]:
# https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
from collections import defaultdict

def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices
        
def modelling_sgk_reg(train, test, feat, select_flg): # stratify by target, group by session_id
    lgbm_params = {'objective': 'regression', 'boosting_type': 'gbdt',
     'tree_learner': 'serial','bagging_fraction': 0.5698056418890787,'bagging_freq': 4,
     'colsample_bytree': 0.37564408454469,'learning_rate': 0.015433389422506185,'max_depth': 8,
     'min_data_in_leaf': 51,'min_sum_hessian_in_leaf': 10,'num_leaves': 48}

    y_train = train.accuracy_group.copy()
    X_train = train.drop(["accuracy_group"],axis=1) 
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["installation_id"]))
    X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
    remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
                      or i in to_exclude]
    for i in X_train.columns:
        if X_train[i].std() == 0 and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    if select_flg == True:
        X_train = X_train[fea]
    X_train = X_train[sorted(X_train.columns.tolist())]

    X_test = test.drop(["installation_id","accuracy_group"], axis=1)
    X_test = X_test.drop(remove_features, axis=1)
    if select_flg == True:
        X_test = X_test[fea]
    X_test = X_test[sorted(X_test.columns.tolist())]
    groups = np.array(X_train.installation_id.values)

    models = []
    random_try = 30
    n_folds = 5
    evals_result = {}
    mean_qwk_score = 0
    for i, (train_index, test_index) in enumerate(stratified_group_k_fold(X_train, y_train, groups, k=n_folds, seed=12)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2.drop("installation_id", axis=1, inplace=True)
        
        for try_time in range(random_try):
            X_test2 = X_train.iloc[test_index,:]
            y_test2 = y_train.iloc[test_index]
            
            # random truncation ------------------
            X_test2, idx_val = get_random_assessment(X_test2)
            X_test2.drop('installation_id', inplace=True, axis=1)
            y_test2 = y_test2.loc[idx_val]
            # ------------------------------------

            lgb_train = lgb.Dataset(X_train2, y_train2)
            lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
            clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500, categorical_feature = categoricals)
            valid = np.array(clf.predict(X_test2, num_iteration = clf.best_iteration))
            real = np.array(y_test2)

            models.append(clf)
            rmse = np.sqrt(mean_squared_error(real, valid))
            print("RMSE = {}".format(rmse))
            
            # threshold optimization --------------
            best_score = 0
            for j in range(20):
                optR = OptimizedRounder_reg()
                optR.fit(np.array(valid).reshape(-1,), real, random_flg=True)
                coefficients = optR.coefficients()
                final_valid_pred = optR.predict(np.array(valid).reshape(-1,), coefficients)
                score = qwk(real, final_valid_pred)
                print(j, np.sort(coefficients), score)
                if score > best_score:
                    best_score = score
                    best_coefficients = coefficients
            mean_qwk_score += best_score / (random_try * n_folds)
            if try_time == 0 and i == 0:
                final_coefficients = np.sort(best_coefficients) / (random_try * n_folds)
            else:
                final_coefficients += np.sort(best_coefficients) / (random_try * n_folds)

    print("mean QWK = {}".format(mean_qwk_score))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return mean_qwk_score, pred_value, final_coefficients
 
feat = []
qwk_sgk1, pred_value, final_coefficients = modelling_sgk_reg(new_train, new_test, feat, False)
final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort(final_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
#final_test_pred = pd.cut(np.array(pred_value).reshape(-1,), [-np.inf] + list(np.sort([0.32229148, 0.51887455, 0.77529457])) + [np.inf], labels = [0, 1, 2, 3])

sample_submission["accuracy_group"] = final_test_pred.astype(int)
sample_submission.to_csv('submission.csv', index=False)
sample_submission["accuracy_group"].value_counts(normalize = True)

Fold 1



Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[307]	training's l2: 0.753044	valid_1's l2: 1.12615
RMSE = 1.061203513908699
0 [1.14302414 1.70067946 2.29130693] 0.52984283
1 [1.11692378 1.62725583 2.2906442 ] 0.52949591
2 [1.11887292 1.77154299 2.135438  ] 0.51813642
3 [1.16913449 1.62195239 2.29031742] 0.53510013
4 [1.0091348  1.69411455 2.29116273] 0.5233329
5 [1.27286695 1.62421195 2.27925555] 0.53997409
6 [1.02194952 1.77071434 2.29111869] 0.52861691
7 [1.33511671 1.64285689 2.29088953] 0.54172993
8 [1.06382173 1.78845768 2.18219211] 0.51452409
9 [1.02095429 1.78528665 2.29067413] 0.52796063
10 [1.01978022 1.80205015 2.29028446] 0.52601547
11 [1.33577961 1.7619713  2.28918501] 0.54277971
12 [1.02163734 1.77056994 2.19625041] 0.51645028
13 [1.19578673 1.76924858 2.29076017] 0.53712491
14 [1.14775476 1.73784605 2.28170495] 0.52934513
15 [1.18847344 1.47343044 2.2901069 ] 0.53098773
16 [1.27934659 1.58576824 2.29130559] 0.54299502
17 


Training until validation scores don't improve for 100 rounds
[500]	training's l2: 0.667523	valid_1's l2: 1.11081
Early stopping, best iteration is:
[457]	training's l2: 0.684375	valid_1's l2: 1.11038
RMSE = 1.0537478620190834
0 [1.10857931 1.7133318  2.16660202] 0.55945217
1 [1.0063988  1.7796014  2.15582711] 0.55887328
2 [1.12646883 1.70998644 2.15128672] 0.55795152
3 [1.10780269 1.75147515 2.1544953 ] 0.55764263
4 [1.12947522 1.71405591 2.15332502] 0.55866868
5 [1.00838273 1.71448779 2.137631  ] 0.56119508
6 [1.00204335 1.75091954 2.13957555] 0.56048886
7 [1.0036464  1.79585271 2.15516412] 0.5599291
8 [1.00161548 1.7968895  2.16646841] 0.56115005
9 [0.99639442 1.75131546 2.140798  ] 0.56048886
10 [1.04380696 1.7515265  2.13812382] 0.55826433
11 [0.99647324 1.75384059 2.15459204] 0.56065041
12 [1.0405935  1.76984385 2.15359443] 0.55722068
13 [0.99579468 1.75621028 2.13719582] 0.56048886
14 [0.9991874  1.78993125 2.1401608 ] 0.55921008
15 [0.99559141 1.75014737 2.20488078] 0.55807081


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[275]	training's l2: 0.775218	valid_1's l2: 1.18189
RMSE = 1.087148555237402
0 [1.09352121 1.75884962 2.27866761] 0.49733765
1 [1.08832403 1.74004398 2.27875962] 0.49690072
2 [1.07468509 1.75963107 2.12404762] 0.49416894
3 [1.07710741 1.73074997 2.20308764] 0.49221984
4 [1.14668275 1.76688091 2.13264296] 0.49730644
5 [1.10513959 1.74516089 2.31286489] 0.49590064
6 [1.06196513 1.73778589 2.29498995] 0.49493433
7 [1.06425792 1.75569343 2.2561911 ] 0.49523181
8 [0.96300273 1.78938814 2.27931917] 0.49638481
9 [1.19430844 1.75604646 2.31740933] 0.50504245
10 [0.96514136 1.76231767 2.27878361] 0.50046476
11 [1.00778823 1.76370034 2.27887021] 0.4962277
12 [1.06904478 1.73353757 2.27904452] 0.49570772
13 [1.05809823 1.72318415 2.27883521] 0.49290285
14 [1.09890203 1.76378899 2.27850581] 0.49851126
15 [1.07564904 1.73516177 2.28523623] 0.49575569
16 [1.09817665 1.7355666  2.12296812] 0.49363094
17 


Training until validation scores don't improve for 100 rounds
[500]	training's l2: 0.667523	valid_1's l2: 1.10481
Early stopping, best iteration is:
[609]	training's l2: 0.633575	valid_1's l2: 1.10219
RMSE = 1.049853199744961
0 [1.03405401 1.77643653 2.21428831] 0.56353173
1 [1.139418   1.77463019 2.16402847] 0.56137492
2 [1.03674551 1.62624268 2.29385004] 0.56346266
3 [1.13237175 1.77565011 2.21064722] 0.5620427
4 [1.14725269 1.63857999 2.21534838] 0.56510535
5 [1.12786853 1.62422575 2.20995903] 0.56365773
6 [1.05515355 1.63667122 2.25016632] 0.56295982
7 [1.05403619 1.77355979 2.21432076] 0.56163074
8 [1.14778544 1.63871149 2.24929965] 0.56532237
9 [1.03654497 1.77712603 2.21435932] 0.56353173
10 [1.18689862 1.6232186  2.17322736] 0.56133701
11 [1.03467328 1.77752492 2.21589898] 0.56280571
12 [1.03468428 1.70227496 2.21433292] 0.55979799
13 [1.07236451 1.62470762 2.21998937] 0.56197854
14 [1.03411753 1.77667047 2.21029672] 0.56319264
15 [1.02676164 1.7777374  2.2143792 ] 0.56353173


...


Training until validation scores don't improve for 100 rounds
[500]	training's l2: 0.657745	valid_1's l2: 1.03757
Early stopping, best iteration is:
[642]	training's l2: 0.614943	valid_1's l2: 1.03441
RMSE = 1.0170618696312346
0 [1.13007933 1.66333378 2.354451  ] 0.60972886
1 [1.10610518 1.65993445 2.3562037 ] 0.60786946
2 [1.0701219  1.65126584 2.30659118] 0.60762842
3 [1.11078942 1.65390076 2.32021118] 0.60728719
4 [1.09415506 1.80100379 2.30587866] 0.60529666
5 [1.0932353  1.64996624 2.14518134] 0.59938522
6 [1.108331   1.79429862 2.1737829 ] 0.59846003
7 [1.11823857 1.66114397 2.32269012] 0.6061259
8 [1.05640849 1.68316864 2.30667845] 0.60424818
9 [1.10564368 1.6543139  2.30610857] 0.60940239
10 [1.12864036 1.66369412 2.30859685] 0.60852789
11 [1.06889761 1.65206228 2.31032931] 0.60671998
12 [1.10192236 1.6537457  2.27345674] 0.60202318
13 [1.11476846 1.70474024 2.30601802] 0.60659221
14 [1.14094317 1.65341828 2.30645286] 0.60842814
15 [1.11794229 1.66025558 2.30663153] 0.6090268



Training until validation scores don't improve for 100 rounds
[500]	training's l2: 0.657745	valid_1's l2: 1.0942
Early stopping, best iteration is:
[511]	training's l2: 0.653236	valid_1's l2: 1.09367
RMSE = 1.0457887534824866
0 [1.13020557 1.8207888  2.2777524 ] 0.56903147
1 [1.00668516 1.78596146 2.24882999] 0.55958878
2 [1.1054588  1.67400548 2.27757744] 0.57220893
3 [1.04197525 1.67934141 2.27808848] 0.5704228
4 [1.15419915 1.48203676 2.24824676] 0.56885402
5 [1.10399576 1.65616584 2.27811318] 0.57256446
6 [1.13564471 1.6640669  2.28062523] 0.5733591
7 [1.02633486 1.82397364 2.27785296] 0.56365323
8 [1.12424264 1.66143985 2.27784196] 0.57406595
9 [1.08533715 1.65389827 2.24827977] 0.57047846
10 [1.12721355 1.82071134 2.27744869] 0.56903147
11 [1.00807952 1.67240545 2.27769365] 0.57155016
12 [1.10424644 1.83918317 2.24779238] 0.56578257
13 [1.13022787 1.8458563  2.27753874] 0.56730198
14 [1.12361606 1.66185917 2.27912413] 0.57326816
15 [1.08024156 1.66482735 2.27750385] 0.57123171
1


Training until validation scores don't improve for 100 rounds
[500]	training's l2: 0.657745	valid_1's l2: 1.01851
Early stopping, best iteration is:
[753]	training's l2: 0.586889	valid_1's l2: 1.01311
RMSE = 1.0065355311153954
0 [1.04913675 1.6077109  2.28906338] 0.60860215
1 [1.08066754 1.60875413 2.24823303] 0.60980754
2 [1.08158915 1.70957995 2.2481464 ] 0.60591391
3 [1.11628663 1.60170986 2.3036633 ] 0.60379628
4 [1.09031066 1.63013171 2.28791551] 0.60503077
5 [1.10133304 1.6153779  2.28925762] 0.60727662
6 [1.10287719 1.61914952 2.25305982] 0.60926804
7 [1.08309198 1.71011041 2.24948505] 0.60591391
8 [1.09661071 1.61957427 2.2537166 ] 0.60922874
9 [1.05144715 1.60792118 2.28824138] 0.60860215
10 [1.07026679 1.61824936 2.24965384] 0.60943471
11 [1.05382593 1.69477598 2.24744372] 0.60592942
12 [1.08127916 1.70452209 2.24997822] 0.60492533
13 [1.05300526 1.67247523 2.25381036] 0.60679037
14 [1.10305163 1.62212496 2.28822282] 0.60652232
15 [1.1031838  1.61776533 2.2541818 ] 0.6092680


Training until validation scores don't improve for 100 rounds
[500]	training's l2: 0.657745	valid_1's l2: 1.12047
Early stopping, best iteration is:
[521]	training's l2: 0.649526	valid_1's l2: 1.11956
RMSE = 1.0580919715934904
0 [1.06512859 1.64797694 2.21701308] 0.55291973
1 [1.05227896 1.82877367 2.21453513] 0.54208718
2 [1.03780096 1.82453675 2.21162182] 0.54214822
3 [1.03908535 1.8352493  2.18460213] 0.54259409
4 [1.06052814 1.8353874  2.19373977] 0.54235082
5 [1.05290698 1.83428672 2.19169396] 0.5428753
6 [1.01872968 1.88725502 2.21789016] 0.54000571
7 [1.0700828  1.64792047 2.21097149] 0.55223452
8 [1.05183067 1.82730654 2.19349753] 0.54196988
9 [1.04093232 1.83491183 2.1851099 ] 0.54259409
10 [1.08002848 1.88471301 2.18556862] 0.54119722
11 [1.09086654 1.82303911 2.17599576] 0.5414306
12 [1.04860196 1.82421615 2.21671921] 0.54398071
13 [1.04987633 1.83370715 2.2045586 ] 0.54280115
14 [1.09408867 1.64877817 2.1876974 ] 0.55189397
15 [1.07203024 1.825294   2.19147235] 0.54215703


3    0.438
2    0.259
0    0.159
1    0.144
Name: accuracy_group, dtype: float64

In [19]:
def my_hyperopt(X, Y):
    def para_tuning_obj(params):
        params = {
        'boosting_type': 'gbdt', 
        'metric': "auc", 
        'objective': 'binary', 
        'eval_metric': 'cappa', 
        "tree_learner": "serial",
        'max_depth': int(params['max_depth']),
        'bagging_freq': int(params['bagging_freq']),
        'bagging_fraction': float(params['bagging_fraction']),
        'num_leaves': int(params['num_leaves']),
        'learning_rate': float(params['learning_rate']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'min_sum_hessian_in_leaf': int(params['min_sum_hessian_in_leaf']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
}
    
        real = np.array([])
        pred = np.array([])
        skf = GroupKFold(n_splits=10)
        for trn_idx, val_idx in skf.split(X, Y, X["installation_id"]):
            x_train, x_val = X.iloc[trn_idx, :], X.iloc[val_idx, :]
            y_train, y_val = Y.iloc[trn_idx], Y.iloc[val_idx]
            x_val['accuracy_group'] = y_val
            np.random.seed(0)
            x_val_mod = x_val.groupby('installation_id').agg(np.random.choice).reset_index(drop=False)
            y_val_mod = x_val_mod.accuracy_group.copy()
            x_train.drop('installation_id', inplace = True, axis = 1)
            x_val_mod.drop(['installation_id', "accuracy_group"], inplace = True, axis = 1)
            train_set = lgb.Dataset(x_train, y_train, categorical_feature = ['session_title'])
            val_set = lgb.Dataset(x_val_mod, y_val_mod, categorical_feature = ['session_title'])
        
            clf = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                         valid_sets = [train_set, val_set], verbose_eval = 300)
            pred = np.concatenate((pred, np.array(clf.predict(x_val_mod, num_iteration = clf.best_iteration))), axis=0) 
            real = np.concatenate((real, np.array(y_val_mod)), axis=0) 
        score = roc_auc_score(real, pred)
    
        return - score

    trials = Trials()

    space ={
        'max_depth': hp.quniform('max_depth', 1, 15, 1),
        'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.2, 1.0),
        'num_leaves': hp.quniform('num_leaves', 8, 64, 1),
        'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
        'min_data_in_leaf': hp.quniform('min_data_in_leaf', 8, 64, 1),
        'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 5, 30, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)
    }

    best = fmin(para_tuning_obj, space = space, algo=tpe.suggest, max_evals=10, trials=trials, verbose=1)

    best_params = space_eval(space, best)
    return best_params

#X_train = new_train.drop(["accuracy_group"], axis=1).copy()
#Y = new_train.accuracy_group.copy()
#Y.loc[Y <=1] = 0
#Y.loc[Y >=2] = 1
#lbl = preprocessing.LabelEncoder()
#lbl.fit(list(X_train["installation_id"]))
#X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
#remove_features = [i for i in X_train.columns if "_4235" in i or i == "world_"+str(activities_world["NONE"])
#                      or i in to_exclude]
#for i in X_train.columns:
#    if X_train[i].std() == 0 and i not in remove_features:
#        remove_features.append(i)
#X_train = X_train.drop(remove_features, axis=1)
#X_train = X_train[sorted(X_train.columns.tolist())]

#random_state = 42
#my_hyperopt(X_train, Y)